In [1]:
from OpticalFlow.FlowFormer.core.FlowFormer import build_flowformer
from OpticalFlow.FlowFormer.configs.faciallandmark import get_cfg
from OpticalFlow.FlowFormer.core.utils.utils import InputPadder
from OpticalFlow.FlowFormer.core.utils.flow_viz import flow_to_image
import torch

cfg=get_cfg()
model=build_flowformer(cfg).to(device='cuda')
#https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/4
from collections import OrderedDict
state_dict=torch.load(cfg.model)
new_state_dict = OrderedDict()
for k, v in state_dict.items():
  name = k[7:] # 删除module.
  new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()
#逐帧处理光流
import cv2
cap=cv2.VideoCapture(0)
cap.open('data/source1.mp4')
lastFrame=None
writer = cv2.VideoWriter("data/flow1.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30., (640,480))
count=0
while cap.isOpened:
  success, frame = cap.read()
  if not success:
    print("End")
    break
  frame=torch.from_numpy(frame).cuda().float().permute([2,0,1]).unsqueeze(0)
  if lastFrame is not None:
    padder = InputPadder(frame.shape)
    with torch.no_grad():
      flow=model(lastFrame,frame)[0].squeeze(0).permute([1,2,0]).cpu().numpy()
      flow_image=flow_to_image(flow)
      cv2.imshow('flow',flow_image)
  lastFrame=frame
  count+=1
  print(count)

cap.release()
writer.release()
cv2.destroyAllWindows()

1


d:\software\TorchStudio\python\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2
3


: 

: 